In [1]:
import numpy as np
import matplotlib.pyplot as plt
import experiment
import interpret
import cosmology
import consensus
import pickle
import copy
import yaml

In [2]:
n_true_sys = 5
true_sys = 1./(np.arange(n_true_sys)+1)**2 * np.random.randn(n_true_sys)
true_sys

array([-0.74554926, -0.24644419, -0.09371895,  0.06109715,  0.00779505])

In [2]:
class Configuration():
    def __init__(self, config_file = None):
        self.config_dict = self.read(config_file)
        pass

    def read(self, config):
        """
        Reads config file into a dictionary and returns it.

        Args:
            config (str): Name of config file.

        Returns:
            config_dict (dict): Dictionary containing config information.
        """

        with open(config, 'r') as config_file_obj:
            config_dict = yaml.safe_load(config_file_obj)

        return config_dict


    def check_configuration(self,config):
        '''
        Checks that the configuration file makes sense, is compliant.
        '''
        necessary_params = ['consensus']#, 'experiment', 'interpreter']
        pass

In [38]:
test = Configuration(config_file='example.yaml')
consensus_sim = test.config_dict['consensus']
experiments = test.config_dict['experiments']

In [35]:
experiment_names = experiments.keys()
for name in experiment_names:
    print(f'Consensus: {name}')
    kwargs = experiments[name]
    print(type(np.array(experiments[name]['true_systematics'])))
    print(np.array(experiments[name]['true_systematics']))

Consensus: experiment A
<class 'numpy.ndarray'>
{'systematics parameter values': [0.0], 'systematics parameter file': None, 'systematics description': 'no true systematics'}
Consensus: experiment B
<class 'numpy.ndarray'>
{'systematics parameter values': [-0.74554926, -0.24644419, -0.09371895, 0.06109715, 0.00779505], 'systematics parameter file': None, 'systematics description': '1./(np.arange(5)+1)**2 * np.random.randn(5) -- (random numbers of gently decreasing size)'}


In [36]:
times = np.array(experiments['experiment A']['experimental parameters']['times'])
print(times)
print(type(times))

[0.   0.1  0.25 0.5  0.66 0.9  1.  ]
<class 'numpy.ndarray'>


In [16]:
times[0]

0.0

In [17]:
experiments['experiment A']['experimental parameters']['times description']

'np.linspace(2.,8.,500)'

In [18]:
experiments['experiment A']['noise_parameters']

[0.03]

In [39]:
[experiments[key].keys() for key in experiments.keys()]

[dict_keys(['class name', 'experimental parameters', 'noise_parameters', 'true_systematics', 'interpreter']),
 dict_keys(['class name', 'experimental parameters', 'noise_parameters', 'true_systematics', 'interpreter'])]

In [40]:
[experiments[key]['class name'] for key in experiments.keys()]

['SimplePendulumExperiment', 'SimplePendulumExperiment']

In [23]:
experiments['experiment A']['class name']

'SimplePendulumExperiment'